<a href="https://colab.research.google.com/github/silveiraluiza/machine-learning/blob/main/Tratamento_de_Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando os dados

In [ ]:
# load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)

def create_X_y_df(df):
  # retrieve the array
  data = df.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  # split into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
  # summarize the shape of the train and test sets
  return X_train, X_test, y_train, y_test 

## Modelo Baseline

O modelo baseline utilizado pelo artigo é uma regressão, onde é previsto um valor numérico para o preço de uma casa. Todas as variáveis de entrada são também numéricas.

A avaliação das previsões é feita utilizando o erro médio absoluto (MAE) no artigo base, portanto iremos seguir utilizando a mesma para comparações.

In [ ]:

X_train, X_test, y_train, y_test = create_X_y_df(df)

# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 3.417


## MAEs dos métodos utilizados no artigo

### Isolation Forest
> MAE: 3.189

### Minimum Covariance Determinant
> MAE: 3.388

### Local Outlier Factor
> MAE: 3.356

### One-Class SVM
> MAE: 3.431


## Z-Score


Em termos simples, o Z-score é uma medida estatística que diz a que distância está um ponto de dados do resto do conjunto de dados. Em um termo mais técnico, o Z-score indica a quantos desvios padrão uma determinada observação está longe da média.

O  Z-score é uma medida paramétrica e requer dois parâmetros - média e desvio padrão. Se o score Z de um ponto de dados for superior a 3, indica que o ponto de dados é bastante diferente dos outros pontos. Ele então pode ser tido como um outlier.


In [ ]:
import scipy.stats as stats
import numpy as np

X_train, X_test, y_train, y_test = create_X_y_df(df)

print("Tamanho do X_train antes de remover os outliers: {}".format(X_train.shape))
z_scores = stats.zscore(X_train)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)

X_train = X_train[filtered_entries]
y_train = y_train[filtered_entries]

print("Tamanho do X_train após remover os outliers: {}".format(X_train.shape))

Tamanho do X_train antes de remover os outliers: (339, 13)
Tamanho do X_train após remover os outliers: (276, 13)


### Treinando o modelo baseline com o dataset sem outliers (Z-Score)

In [ ]:

# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
y_pred = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

MAE: 3.263


Usando a abordagem do Z-Score, obtivemos uma MAE de 3.270, que é menor do que a do modelo baseline e menor também do que a maioria das MAEs encontradas, menos a do método de isolation forest.

## DBScan

DBSCAN é uma abordagem de agrupamento com base na densidade, e não um método de identificação de anomalias por si só. Ele desenvolve agrupamentos com base numa medida de distância. Os pontos centrais - pontos que têm um mínimo de pontos no seu redor - e pontos que estão suficientemente próximos desses pontos centrais juntos formam um grupo.

Podemos usar DBSCAN como um algoritmo de detecção de outlier porque os pontos que não pertencem a nenhum aglomerado adquirem a sua própria classe: -1. O algoritmo tem dois parâmetros (epsilon: escala de comprimento, e min_sample: o número mínimo de amostras necessárias para que um ponto seja um ponto central). Encontrar um bom epsilon é crítico.

DBSCAN faz portanto previsões binárias: um ponto ou é um outlier ou não. Para refinar as previsões, consideramos os outros aglomerados para além do aglomerado principal, também como aglomerados outliers, quanto menor for o aglomerado, maior será a pontuação outliers.

A função de distância utilizada é a distância euclidiana padrão. Note-se que o pior desempenho do DBSCAN é O(n^2), se a varredura de vizinhança for uma varredura linear, o que é o caso da implementação do sci-kit learn implementation. Isto limita significativamente o tamanho do conjunto de dados que pode ser analisado.


In [ ]:
from sklearn.cluster import DBSCAN
from numpy import random, where

mae_list = []
eps_samples = []

# Variamos o epsilon e o min_sample para obter melhores resultados

for eps in (0.3, 0.5, 1, 5, 10, 20, 30, 40, 50):
  for min_samples in (3, 5, 10, 15, 20, 30):
    dbscan = DBSCAN(eps = eps, min_samples = min_samples)
    X_train, X_test, y_train, y_test = create_X_y_df(df)
    pred = dbscan.fit_predict(X_train)
    index_non_anom = where(pred != -1)

    if index_non_anom[0].shape[0] > 200:
      mask = index_non_anom[0]
      
      X_train, y_train = X_train[mask, :], y_train[mask]
      # fit the model
      model = LinearRegression()
      model.fit(X_train, y_train)
      # evaluate the model
      y_pred = model.predict(X_test)
      # evaluate predictions
      mae = mean_absolute_error(y_test, y_pred)
      eps_samples.append((eps,min_samples))
      mae_list.append(mae)
      print("--------------------------")
      print(mae)
      print((eps,min_samples))
      print(filtered_entries.shape)

print("--------------------------")
print("Menor MAE")
print(min(mae_list))


--------------------------
3.420371501345386
(20, 3)
(339,)
--------------------------
3.5382556600347517
(20, 5)
(339,)
--------------------------
3.2664418344507906
(30, 3)
(339,)
--------------------------
3.3096989638552423
(30, 5)
(339,)
--------------------------
3.663021654401295
(30, 10)
(339,)
--------------------------
8.155231037588123
(30, 15)
(339,)
--------------------------
3.429035959630473
(40, 3)
(339,)
--------------------------
3.401218051788704
(40, 5)
(339,)
--------------------------
3.3023915761425924
(40, 10)
(339,)
--------------------------
4.556643048942376
(40, 15)
(339,)
--------------------------
5.283978798472505
(40, 20)
(339,)
--------------------------
5.723088356783139
(40, 30)
(339,)
--------------------------
3.4202931253955033
(50, 3)
(339,)
--------------------------
3.4201103145642904
(50, 5)
(339,)
--------------------------
3.380718799459045
(50, 10)
(339,)
--------------------------
3.3003416735538056
(50, 15)
(339,)
-------------------------

In [ ]:
index = mae_list.index(3.2664418344507906)
eps_samples[index]

(30, 3)

### Treinando o modelo baseline com o dataset sem outliers (DBSCAN)

In [ ]:
dbscan = DBSCAN(eps = 30, min_samples = 3)
X_train, X_test, y_train, y_test = create_X_y_df(df)
print("Tamanho do X_train antes de remover os outliers: {}".format(X_train.shape))

pred = dbscan.fit_predict(X_train)
index_non_anom = where(pred != -1)
mask = index_non_anom[0]

X_train, y_train = X_train[mask, :], y_train[mask]
print("Tamanho do X_train após remover os outliers: {}".format(X_train.shape))
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
y_pred = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

Tamanho do X_train antes de remover os outliers: (339, 13)
Tamanho do X_train após remover os outliers: (307, 13)
MAE: 3.266


Usando a abordagem do DBSCAN, obtivemos uma MAE de 3.266, que é menor do que a do modelo baseline e, assim como o Z-score, menor também do que a maioria das MAEs encontradas, menos a do método de isolation forest. Ela é bastante similar a MAE do Z-Score, sendo maior apenas por 0.003 pontos, no entanto remove menos dados e obtém um resultado bastante semelhante.